In [1]:
import pandas as pd
import sqlite3
import numpy as np
roots_tfm = '/home/julien/Documents/Etudes/Astrofisica/Master/TFM/Data'

In [2]:
con = sqlite3.connect(roots_tfm + '/UVCatalogue.db')

In [3]:
cursor = con.cursor()
keys = {'SRCNUM_1', 'SRCNUM_2', 'RA', 'DEC'}
sql_command = "SELECT SRCNUM_1, SRCNUM_2, RA, DEC from SOURCES where COUNT>1"
sources_selection = cursor.execute(sql_command).fetchall()
cursor = con.cursor()
con.close()

In [4]:
results = []
for source_ind in range(len(sources_selection)):
    entries_dict ={}
    for j,key in enumerate(keys):
        entries_dict[key] = sources_selection[source_ind][j]
    results.append(entries_dict)
df_sources = pd.DataFrame(results, columns=[key for key in keys])        

In [5]:
df_sources['SRCNUM'] = range(df_sources.shape[0])
df_sources.head()

,SRCNUM_1,RA,SRCNUM_2,DEC,SRCNUM
0,1,5,0.000164,8.371723,0
1,3,0,0.000804,81.916291,1
2,4,0,0.001279,-77.007358,2
3,9,0,0.004545,-76.985881,3
4,11,102,0.005971,8.163872,4


In [6]:
df_entries_uvot = pd.read_csv(roots_tfm + '/Swift-UVOT/Entries_UVOT_csv',usecols = ['SRCNUM','UVM2_FLUX','UVW1_FLUX','UVW2_FLUX','MJD'])
df_entries_uvot = df_entries_uvot[df_entries_uvot['SRCNUM'].isin(df_sources['SRCNUM_1'])].dropna(how='all',subset=['UVM2_FLUX','UVW1_FLUX','UVW2_FLUX'])
df_entries_uvot = df_entries_uvot.rename(columns={'SRCNUM': 'SRCNUM_1'})
df_entries_uvot.shape

(4713657, 5)

In [7]:
df_entries_om = pd.read_csv(roots_tfm + '/XMM-Newton/Entries_OM_csv',usecols = ['SRCNUM','UVM2_AB_FLUX','UVW1_AB_FLUX','UVW2_AB_FLUX','MJD_START'])
df_entries_om = df_entries_om[df_entries_om['SRCNUM'].isin(df_sources['SRCNUM_2'])].dropna(how='all',subset=['UVM2_AB_FLUX','UVW1_AB_FLUX','UVW2_AB_FLUX'])
df_entries_om = df_entries_om.rename(columns={'SRCNUM': 'SRCNUM_2','UVM2_AB_FLUX' : 'UVM2_FLUX','UVW1_AB_FLUX': 'UVVW1_FLUX',\
                                              'UVW2_AB_FLUX':'UVW2_FLUX','MJD_START': 'MJD'})
df_entries_om.shape

(0, 5)

In [8]:
df_entries_uvot = df_entries_uvot.merge(df_sources, how='left',on='SRCNUM_1').drop({'SRCNUM_1', 'SRCNUM_2','RA','DEC'},axis=1)
df_entries_uvot.head()

,UVW2_FLUX,UVM2_FLUX,UVW1_FLUX,MJD,SRCNUM
0,7.057068e-16,1.439141e-16,NaN,54973.746285,0
1,1.230489e-16,2.194885e-16,NaN,55413.664630,2
2,NaN,1.822581e-16,NaN,55438.008171,2
3,NaN,NaN,5.510053e-16,55467.000856,2
4,NaN,5.458706e-17,NaN,55438.008171,3


In [9]:
df_entries_om = df_entries_om.merge(df_sources, how='left',on='SRCNUM_2').drop({'SRCNUM_1', 'SRCNUM_2','RA','DEC'},axis=1)
df_entries_om.head()

,UVW2_FLUX,UVM2_FLUX,UVVW1_FLUX,MJD,SRCNUM


In [10]:
df_entries = pd.concat([df_entries_om,df_entries_uvot]).reset_index()
df_entries.shape

(4713657, 7)

In [17]:
def variability(group):

    variability_dict ={}
    for flux in ['UVW2_FLUX','UVM2_FLUX','UVW1_FLUX']:  
        if group[flux].count()>1:
            idmax = group[flux].idxmax()
            idmin = group[flux].idxmin()
            variability_dict[flux] = 100 * (group.loc[idmax,flux]-group.loc[idmin,flux])/(group.loc[idmax,flux]* (group.loc[idmax,'MJD'] - group.loc[idmin,'MJD']))
        else:
            variability_dict[flux] = np.nan
    return pd.Series({
        'VAR_UVW2': variability_dict['UVW2_FLUX'],
        'VAR_UVM2': variability_dict['UVM2_FLUX'],
        'VAR_UVW1': variability_dict['UVW1_FLUX']
    })  

In [12]:
df_entries_groups = df_entries.groupby('SRCNUM')

In [13]:
u=0
for name, group in df_entries_groups:
    variability(group)
    u=u+1
    if u>1:
        break

dict{'UVW2_FLUX': nan, 'UVM2_FLUX': nan, 'UVW1_FLUX': nan}
indmax1
indmax2
flux max2.1948854e-16
flux min1.8225809000000003e-16
date max55413.66462962963
date min55438.0081712963
dict{'UVW2_FLUX': nan, 'UVM2_FLUX': np.float64(-1.529376888120425e-18), 'UVW1_FLUX': nan}


In [ ]:
df_entries_source = df_entries.groupby('SRCNUM').apply(variability) 

/tmp/ipykernel_6465/3908416935.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  variability_dict[flux] = 100 * (group.loc[idmax,flux]-group.loc[idmin,flux])/(group.loc[idmax,flux]* (group.loc[idmax,'MJD'] - group.loc[idmin,'MJD']))
/tmp/ipykernel_6465/3908416935.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  variability_dict[flux] = 100 * (group.loc[idmax,flux]-group.loc[idmin,flux])/(group.loc[idmax,flux]* (group.loc[idmax,'MJD'] - group.loc[idmin,'MJD']))
/tmp/ipykernel_6465/3908416935.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  variability_dict[flux] = 100 * (group.loc[idmax,flux]-group.loc[idmin,flux])/(group.loc[idmax,flux]* (group.loc[idmax,'MJD'] - group.loc[idmin,'MJD']))
/tmp/ipykernel_6465/3908416935.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  variability_dict[flux] = 100 * (group.loc[idmax,flux]-group.loc[idmin,flux])/(group.loc[idmax,flux]* (group.loc[idmax,'MJD'] - group.loc[idmin,'MJD'

In [15]:
#df_entries['COUNT'] = df_entries['SRCNUM'].copy()
#df_entries['UVW2_FLUX_IDXMIN'] = df_entries['UVW2_FLUX'].copy()
#df_entries['UVW2_FLUX_IDXMAX'] = df_entries['UVW2_FLUX'].copy()
#df_entries['UVM2_FLUX_IDXMIN'] = df_entries['UVM2_FLUX'].copy()
#df_entries['UVM2_FLUX_IDXMAX'] = df_entries['UVM2_FLUX'].copy()
#df_entries['UVW1_FLUX_IDXMIN'] = df_entries['UVW1_FLUX'].copy()
#df_entries['UVW1_FLUX_IDXMAX'] = df_entries['UVW1_FLUX'].copy()
#df_entries_source = df_entries.groupby('SRCNUM').agg({'COUNT': 'count',
#                                          'UVW2_FLUX_IDXMIN': 'idxmin', 'UVW2_FLUX_IDXMAX':'idxmax',
#                                          'UVM2_FLUX_IDXMIN': 'idxmin', 'UVM2_FLUX_IDXMAX':'idxmax',
#                                          'UVW1_FLUX_IDXMIN': 'idxmin', 'UVW1_FLUX_IDXMAX':'idxmax',})
#ind_NaN_UVW2 = df_entries_source['UVW2_FLUX_IDXMIN']==np.nan
#ind_NaN_UVW1 = df_entries_source['UVW1_FLUX_IDXMIN']==np.nan
#ind_NaN_UVM2 = df_entries_source['UVM2_FLUX_IDXMIN']==np.nan
    
# Computation of the variability
#df_entries_source.loc[~ind_NaN_UVW2,'UVW2_VAR'] = (df_entries.loc[df_entries_source.loc[~ind_NaN_UVW2,'UVW2_FLUX_IDXMAX'],'UVW2_FLUX'] \
#                                                   - df_entries.loc[df_entries_source.loc[~ind_NaN_UVW2,'UVW2_FLUX_IDXMIN'],'UVW2_FLUX']) \
#/ (df_entries.loc[df_entries_source.loc[~ind_NaN_UVW2,'UVW2_FLUX_IDXMAX'],'MJD'] \
#   - df_entries.loc[df_entries_source.loc[~ind_NaN_UVW2,'UVW2_FLUX_IDXMIN'],'MJD'])
#df_entries_source.loc[~ind_NaN_UVW1,'UVW1_VAR'] = (df_entries.loc[df_entries_source.loc[~ind_NaN_UVW1,'UVW1_FLUX_IDXMAX'],'UVW1_FLUX'] \
#                                                   - df_entries.loc[df_entries_source.loc[~ind_NaN_UVW1,'UVW1_FLUX_IDXMIN'],'UVW1_FLUX']) \
#/ (df_entries.loc[df_entries_source.loc[~ind_NaN_UVW1,'UVW1_FLUX_IDXMAX'],'MJD'] \
#   - df_entries.loc[df_entries_source.loc[~ind_NaN_UVW1,'UVW1_FLUX_IDXMIN'],'MJD'])
#df_entries_source.loc[~ind_NaN_UVM2,'UVM2_VAR'] = (df_entries.loc[df_entries_source.loc[~ind_NaN_UVM2,'UVM2_FLUX_IDXMAX'],'UVM2_FLUX'] -\
#                                                   df_entries.loc[df_entries_source.loc[~ind_NaN_UVM2,'UVM2_FLUX_IDXMIN'],'UVM2_FLUX']) \
#/ (df_entries.loc[df_entries_source.loc[~ind_NaN_UVM2,'UVM2_FLUX_IDXMAX'],'MJD'] \
#   - df_entries.loc[df_entries_source.loc[~ind_NaN_UVM2,'UVM2_FLUX_IDXMIN'],'MJD'])

In [16]:
#list_src_uvot = [srcnum_1 for srcnum_1 in df_sources['SRCNUM_1'] if srcnum_1>0]
#list_src_om = [srcnum_2 for srcnum_2 in df_sources['SRCNUM_2'] if srcnum_2>0]
#import numpy as np
#n=20
#list_src_uvot_chunk = [list_src_uvot[i:i+n] for i in range(0,len(list_src_uvot),n)]
#list_src_om_chunk = [list_src_om[i:i+n] for i in range(0,len(list_src_om),n)]
#import time
#start = time.time()
#ans_uvot = []
#for chunk in list_src_uvot_chunk:
#    sql_command = "SELECT SRCNUM, MJD, UVW2_FLUX, UVW1_FLUX, UVM2_FLUX from ENTRIES_UVOT where SRCNUM IN " \
#    + str(tuple(chunk))
#    ans_uvot.append(cursor.execute(sql_command).fetchall())
#for chunk in list_src_om_chunk:
#    sql_command = "SELECT SRCNUM, MJD, UVW2_FLUX, UVW1_FLUX, UVM2_FLUX from ENTRIES_OM where SRCNUM IN " \
#    + str(tuple(chunk))
#    ans_om.append(cursor.execute(sql_command).fetchall())
#end = time.time()
#print(end - start)